<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.sofisa.com.br/wp-content/uploads/2018/05/new-sofisa.com_.br_.svg" alt="Sofisa" width="220" align="right" />

# FIAP Challenge Sofisa - Demonstração do Teradata Vantage
    
</header>

Cesar Augusto da Silva - educador.cesar@gmail.com

Lucas Alexandre Alves da Silva - lucas.alexandre1@hotmail.com

Marivaldo dos Santos Barbosa - marivaldo.barbosa@icloud.com

Thiago Alberto de Souza Colen - thiago.souzacolen@gmail.com


Como protótipo da utilização da arquitetura escolhida pela equipe, vamos demonstrar neste Jupyter Notebook um exemplo de utilização do Teradata Vantage e sua facilidade em gerar insights através de consultas à dados digitais, armazenados em Object Storage e também a partir de algumas de suas funcionalidades, como por exemplo a função NPATH (https://docs.teradata.com/r/aKnWloeEx3pmus0aivWOaw/MOSRYZ0og7DX6xFvY5X5KA).

A função nPath verifica um conjunto de linhas, procurando os padrões que você especifica. Para cada conjunto de linhas de entrada que correspondem ao padrão, nPath produz uma única linha de saída. A função fornece um recurso flexível de correspondência de padrões que permite especificar padrões complexos nos dados de entrada e definir os valores de saída para cada conjunto de entrada correspondido.


Como funciona a função nPath?

nPath é útil quando seu objetivo é identificar os caminhos que levam a um resultado. Por exemplo, você pode usar o nPath para analisar:

Dados de cliques em sites, para identificar caminhos que levam a vendas acima de um valor especificado

Dados de sensores de processos industriais, para identificar caminhos para a baixa qualidade do produto

Registros de saúde de pacientes individuais, para identificar caminhos que indicam que os pacientes estão em risco de desenvolver doenças como doenças cardíacas ou diabetes

Dados financeiros para indivíduos, para identificar caminhos que fornecem informações sobre riscos de crédito ou fraude


Neste exemplo, vamos utilizar o Teradata Vantage para consultar os eventos que os correntistas tomam em seu Internet Banking e que acabam os levando à entrar em contato com o Customer Support.
Em um segundo exemplo, vamos identificar dentro de um outro dataset, qual o caminho que correntistas tomam, em suas interações com o banco, até a busca de um Consultor Financeiro para gerir seu patrimônio, no caso, de alto padrão financeiro. 

Utilizaremos este Notebook para realizar as consultas ao banco de dados porém, executaremos a visualização desses resultados através do PowerBI.


---

1) Primeramente, vamos realizar nossa conexão ao banco de dados Teradata Vantage.

In [3]:
%connect TRDT01

User Name: nos_usr
Password: ·······


Success: 'TRDT01' connection established and activated for user 'nos_usr'


2) Agora, como buscamos dados digitais em Object Store existente no Azure (Blob Storage / ADLS) vamos criar uma Autorização para que nosso usuário tenha acesso à tal Contêiner.

<a href="https://ibb.co/xHKLNPm"><img src="https://i.ibb.co/q5LjvX0/Container.png" alt="Container" border="0"></a>

In [37]:
CREATE AUTHORIZATION DefAuth_AZ
AS DEFINER TRUSTED
USER 'sofisachallenge' /* storage account name */
PASSWORD 'x5Szh3fFXdwCbIEAnMc5ICBkMvu4bEA6B0jbJJ7jUlWTxLwgNxXWYAvQ1VdCzvZMzfOdsyPInLh3WQpjjml1bQ==' ; /* storage account key */

Success: 0 rows affected

3) Vamos criar uma Foreign Table, que utilizando a autorização criada acima, irá buscar dados CSV, existentes dentro de nosso Conteiner. O Teradata Vantage possibilita a leitura de arquivos CSV, JSON e PARQUET existentes em um Object Storage.

In [54]:
CREATE MULTISET FOREIGN TABLE ft_bank_web_clicks ,FALLBACK ,EXTERNAL SECURITY DEFINER TRUSTED DefAuth_AZ,
     MAP = TD_MAP1
( Location VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC,
Payload DATASET INLINE LENGTH 64000 STORAGE FORMAT CSV
)
USING
(
      LOCATION  ('/AZ/sofisachallenge.blob.core.windows.net/internetbanking/')
    )
    ;

Success: 0 rows affected

4) Vamos analisar duas linhas de payload existente dentro deste Conteiner

In [55]:
SELECT TOP 2 payload
FROM ft_bank_web_clicks;

,Payload
1,"customer_id,session_id,page,datestamp 529,0,FAQ,17/03/2004 16:38:30.000000"
2,"customer_id,session_id,page,datestamp 529,0,ACCOUNT SUMMARY,17/03/2004 16:35:59.000000"


5) Podemos com a consulta abaixo, listar quais são as CSVKEYS de nosso dataset. Isso nos auxiliará nos próximos passos para tabelar esse CSV

In [56]:
SELECT DISTINCT * FROM DATASET_KEYS (ON (SELECT payload FROM
ft_bank_web_clicks)) AS
csvKeys;

,DatasetKeys
1,datestamp
2,session_id
3,page
4,customer_id


5) Neste momento, podemos já executar a função nPath diretamente contra o conteiner listado. Nossa consulta abaixo, irá tabelar os dados do arquivo CSV e, em tempo de execução, utilizar a tabela dentro da função nPath.
De maneira simples, o resultado da função será o caminho percorrido pelos correntistas, dentro do Internet Banking, até a página de Customer Support. A coluna "cnt" corresponde a quantidade de vezes que esse caminho foi percorrido pelos diversos usuários do Internet Banking.

In [71]:
SELECT path, count(*) as cnt
FROM nPath
(ON (SELECT CAST(payload..customer_id AS INTEGER) customer_id,
CAST(payload..session_id AS INTEGER) session_id,
CAST(payload..page AS VARCHAR(100) CHARACTER SET LATIN NOT CASESPECIFIC) page,
CAST(payload..datestamp AS TIMESTAMP(6) FORMAT 'DD-MM-YYYYBHH:MI:SSDS(F)Z') datestamp
FROM ft_bank_web_clicks)
PARTITION BY customer_id, SESSION_ID 
ORDER BY datestamp
USING
Mode (NONOVERLAPPING)
Pattern ('PAGE{1,4}.CSP')
Symbols (TRUE AS PAGE,
page = 'CUSTOMER SUPPORT' AS CSP)
Result (Accumulate(page OF ANY (CSP, page)) as path)
) AS dt GROUP BY path HAVING cnt  >= 11 ORDER BY cnt desc;


,path,cnt
1,"[ACCOUNT SUMMARY, CUSTOMER SUPPORT]",6711
2,"[ACCOUNT SUMMARY, ACCOUNT HISTORY, CUSTOMER SUPPORT]",1157
3,"[ACCOUNT SUMMARY, ACCOUNT SUMMARY, CUSTOMER SUPPORT]",1034
4,"[ACCOUNT SUMMARY, FUNDS TRANSFER, CUSTOMER SUPPORT]",737
5,"[ACCOUNT SUMMARY, FAQ, CUSTOMER SUPPORT]",691
6,"[ACCOUNT SUMMARY, VIEW DEPOSIT DETAILS, CUSTOMER SUPPORT]",652
7,"[ACCOUNT SUMMARY, ONLINE STATEMENT ENROLLMENT, CUSTOMER SUPPORT]",595
8,"[ACCOUNT SUMMARY, PROFILE UPDATE, CUSTOMER SUPPORT]",572
9,"[ACCOUNT SUMMARY, VIEW DEPOSIT DETAILS, ACCOUNT HISTORY, CUSTOMER SUPPORT]",290
10,"[ACCOUNT SUMMARY, FUNDS TRANSFER, ACCOUNT HISTORY, CUSTOMER SUPPORT]",225


6) Se for de nosso interesse, podemos materializar a tabela antes de executar a função nPath. Veja que o resultado da função é o mesmo.

In [61]:
CREATE MULTISET TABLE bank_web_clicks AS (
SELECT CAST(payload..customer_id AS INTEGER) customer_id,
CAST(payload..session_id AS INTEGER) session_id,
CAST(payload..page AS VARCHAR(100) CHARACTER SET LATIN NOT CASESPECIFIC) page,
CAST(payload..datestamp AS TIMESTAMP(6) FORMAT 'DD-MM-YYYYBHH:MI:SSDS(F)Z') datestamp
FROM ft_bank_web_clicks
)
WITH DATA
PRIMARY INDEX ( customer_id );

Success: 0 rows affected

In [72]:
SELECT path, count(*) as cnt
FROM nPath
(ON bank_web_clicks 
PARTITION BY customer_id, SESSION_ID 
ORDER BY datestamp
USING
Mode (NONOVERLAPPING)
Pattern ('PAGE{1,4}.CSP')
Symbols (TRUE AS PAGE,
page = 'CUSTOMER SUPPORT' AS CSP)
Result (Accumulate(page OF ANY (CSP, page)) as path)
) AS dt GROUP BY path HAVING cnt  >= 11 ORDER BY cnt desc;


,path,cnt
1,"[ACCOUNT SUMMARY, CUSTOMER SUPPORT]",6711
2,"[ACCOUNT SUMMARY, ACCOUNT HISTORY, CUSTOMER SUPPORT]",1157
3,"[ACCOUNT SUMMARY, ACCOUNT SUMMARY, CUSTOMER SUPPORT]",1034
4,"[ACCOUNT SUMMARY, FUNDS TRANSFER, CUSTOMER SUPPORT]",737
5,"[ACCOUNT SUMMARY, FAQ, CUSTOMER SUPPORT]",691
6,"[ACCOUNT SUMMARY, VIEW DEPOSIT DETAILS, CUSTOMER SUPPORT]",652
7,"[ACCOUNT SUMMARY, ONLINE STATEMENT ENROLLMENT, CUSTOMER SUPPORT]",595
8,"[ACCOUNT SUMMARY, PROFILE UPDATE, CUSTOMER SUPPORT]",572
9,"[ACCOUNT SUMMARY, VIEW DEPOSIT DETAILS, ACCOUNT HISTORY, CUSTOMER SUPPORT]",290
10,"[ACCOUNT SUMMARY, FUNDS TRANSFER, ACCOUNT HISTORY, CUSTOMER SUPPORT]",225


In [ ]:
---

No exemplo abaixo, vamos executar a função nPath contra um dataset maior. Neste caso, desejamos encontrar  qual o caminho que correntistas tomam, em suas interações com o banco, até a busca de um Consultor Financeiro para gerir seu patrimônio, no caso, de alto padrão financeiro. 

Vamos analisar um amostra de 10 linhas deste dataset

In [30]:
SELECT customer_identifier, interaction_timestamp, interaction_type, product_category, interaction_type || '_' || product_category AS event, 
                marketing_category, marketing_description, marketing_placement, sales_channel, 
                conversion_sales, conversion_cost, conversion_margin
            FROM NOS_USR.ich_banking SAMPLE 10;

,customer_identifier,interaction_timestamp,interaction_type,product_category,event,marketing_category,marketing_description,marketing_placement,sales_channel,conversion_sales,conversion_cost,conversion_margin
1,695117528xdmrpxt390016463,2016-02-26 18:47:41.000000-07:00,BROWSE,MORTGAGE,BROWSE_MORTGAGE,-1,\N,-1,-1,-1,-1,-1
2,705162011ucdhowq550823887,2016-02-09 23:51:43.000000-07:00,BROWSE,AUTO LOAN,BROWSE_AUTO LOAN,-1,\N,-1,-1,-1,-1,-1
3,313591543ljyzmub185932883,2016-03-12 18:28:06.000000-07:00,CLICK,CD,CLICK_CD,OFFSITE_DISPLAY_ADVERTISING,MONEYSUPERMARKET.COM PROMOTION,MONEYSUPERMARKET.COM,-1,-1,-1,-1
4,822096043pjeonlj902248054,2016-01-05 18:59:48.000000-07:00,CLICK,CREDIT CARD,CLICK_CREDIT CARD,SEARCH_ADVERTISING,PLATINUM CARD PROMOTION,GOOGLE,-1,-1,-1,-1
5,503474559ettjrhh914843574,2016-02-09 04:13:58.000000-07:00,COMPARE,CD,COMPARE_CD,-1,\N,-1,-1,-1,-1,-1
6,448535952nsscqwo660956167,2016-01-15 17:01:10.000000-07:00,BROWSE,BROKERAGE,BROWSE_BROKERAGE,-1,\N,-1,-1,-1,-1,-1
7,511411935bsfamlg851955772,2016-03-14 16:02:15.000000-07:00,COMPLETE_APPLICATION,-1,COMPLETE_APPLICATION_-1,-1,\N,-1,-1,-1,-1,-1
8,287707283phuhdqn646619015,2016-02-09 17:28:13.000000-07:00,BROWSE,SAVINGS,BROWSE_SAVINGS,-1,\N,-1,-1,-1,-1,-1
9,214906976tdhjgpd178495968,2016-03-25 15:09:36.000000-07:00,COMPARE,CD,COMPARE_CD,-1,\N,-1,-1,-1,-1,-1
10,945479006wrcosbv969956658,2016-03-25 15:00:37.000000-07:00,BROWSE,SECURED LOAN,BROWSE_SECURED LOAN,-1,\N,-1,-1,-1,-1,-1


Agora vamos executar a função nPATH e analisar o resultado. 

In [42]:
SELECT interaction_type_list,count(*) FROM nPath (
        ON (
	    SELECT customer_identifier, interaction_timestamp, interaction_type, product_category, interaction_type || '_' || product_category AS event, 
                marketing_category, marketing_description, marketing_placement, sales_channel, 
                conversion_sales, conversion_cost, conversion_margin
            FROM NOS_USR.ich_banking
            WHERE
                product_category <> '-1'
                AND interaction_type || '_' || product_category <> 'STARTS_APPLICATION_WEALTH MANAGEMENT'
                AND interaction_type || '_' || product_category <> 'COMPLETE_APPLICATION_WEALTH MANAGEMENT'
        )
        PARTITION BY customer_identifier
        ORDER BY interaction_timestamp
        USING
        MODE (NONOVERLAPPING)
        -- Limit to a depth of 4
        PATTERN ('(EVENT){4}.ADOPTION')
        SYMBOLS (
            event NOT LIKE 'ACCOUNT_BOOKED%_WEALTH MANAGEMENT' AS EVENT,
            event LIKE 'ACCOUNT_BOOKED%' AND product_category = 'WEALTH MANAGEMENT' AS ADOPTION
        )
        RESULT (
            ACCUMULATE( event OF ANY(EVENT,ADOPTION) )  AS interaction_type_list
    )
    )a group by 1 order by 2 desc;

,interaction_type_list,Count(*)
1,"[COMPLETE_APPLICATION_CD, ACCOUNT_BOOKED_ONLINE_CD, BROWSE_WEALTH MANAGEMENT, COMPARE_WEALTH MANAGEMENT, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",40
2,"[COMPLETE_APPLICATION_SAVINGS, ACCOUNT_BOOKED_ONLINE_SAVINGS, COMPARE_WEALTH MANAGEMENT, ENROLL_AUTO_SAVINGS_CHECKING, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",27
3,"[COMPLETE_APPLICATION_BROKERAGE, ACCOUNT_BOOKED_ONLINE_BROKERAGE, BROWSE_WEALTH MANAGEMENT, COMPARE_WEALTH MANAGEMENT, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",21
4,"[COMPLETE_APPLICATION_CD, ACCOUNT_BOOKED_ONLINE_CD, ADD_DIRECT_DEPOSIT_CHECKING, ENROLL_AUTO_SAVINGS_CHECKING, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",16
5,"[COMPLETE_APPLICATION_CD, COMPLETE_APPLICATION_CD, ACCOUNT_BOOKED_ONLINE_CD, BROWSE_WEALTH MANAGEMENT, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",16
6,"[COMPLETE_APPLICATION_BROKERAGE, ACCOUNT_BOOKED_ONLINE_BROKERAGE, ADD_DIRECT_DEPOSIT_CHECKING, ENROLL_AUTO_SAVINGS_CHECKING, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",15
7,"[COMPLETE_APPLICATION_SAVINGS, ACCOUNT_BOOKED_ONLINE_SAVINGS, BROWSE_WEALTH MANAGEMENT, COMPARE_WEALTH MANAGEMENT, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",13
8,"[COMPLETE_APPLICATION_BROKERAGE, COMPLETE_APPLICATION_BROKERAGE, ACCOUNT_BOOKED_ONLINE_BROKERAGE, BROWSE_WEALTH MANAGEMENT, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",10
9,"[STARTS_APPLICATION_CD, COMPLETE_APPLICATION_CD, ACCOUNT_BOOKED_ONLINE_CD, BROWSE_WEALTH MANAGEMENT, ACCOUNT_BOOKED_ONLINE_WEALTH MANAGEMENT]",10
10,"[COMPLETE_APPLICATION_CD, ACCOUNT_BOOKED_ONLINE_CD, BROWSE_WEALTH MANAGEMENT, REFERRAL_WEALTH MANAGEMENT, ACCOUNT_BOOKED_OFFLINE_WEALTH MANAGEMENT]",8


### Power BI

Vamos utilizar os dois exemplos acima e nos conectar ao banco através do PowerBI e executar as funções através dele!